In [1]:
import pandas as pd

In [4]:
df_train = pd.read_csv('../Data/df_train.csv')
df_val = pd.read_csv('../Data/df_val.csv')

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import albumentations as A

In [6]:
def unet_model(input_shape, num_classes):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = concatenate([up6, drop4], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv3], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv2], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, conv1], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(conv9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

input_shape = (256, 256, 3)  # Taille d'entrée de l'image
num_classes = 8
model = unet_model(input_shape, num_classes)

In [8]:
transform = A.Compose([
    A.HorizontalFlip(p=1.0)
])

In [9]:
input_shape2 = (256, 256, 3)

def load_data(df):
    images_tab = []
    masks_tab = []
    for i in range(len(df)):
        image_path = df['Image_Path'][i]
        mask_path = df['Target_Path'][i]
        image = img_to_array(Image.open(image_path).convert('RGB').resize(input_shape[:2]))
        image = image.astype(np.uint8)
        images_tab.append(image)

        mask = img_to_array(Image.open(mask_path).resize(input_shape2[:2], Image.NEAREST))
        mask = mask.astype(np.uint8)
        for x in range(mask.shape[0]):
            for y in range(mask.shape[1]):
                if (mask[x, y] == [250, 170, 30]).all():
                    mask[x, y] = 0
                elif (mask[x, y] == [0, 0, 142]).all():
                    mask[x, y] = 1
                elif (mask[x, y] == [102, 102, 156]).all():
                    mask[x, y] = 2
                elif (mask[x, y] == [220, 20, 60]).all():
                    mask[x, y] = 3
                elif (mask[x, y] == [153, 153, 153]).all():
                    mask[x, y] = 4
                elif (mask[x, y] == [244, 35, 232]).all():
                    mask[x, y] = 5
                elif (mask[x, y] == [70, 70, 70]).all():
                    mask[x, y] = 6
                elif (mask[x, y] == [70, 130, 180]).all():
                    mask[x, y] = 7
        mask = mask[:, :, 0:1]
        masks_tab.append(mask)

        transformed = transform(image=image, mask=mask)
        transformed_img = transformed['image']
        images_tab.append(transformed_img)

        transformed_img = transformed['mask']
        for x in range(transformed_img.shape[0]):
            for y in range(transformed_img.shape[1]):
                if (transformed_img[x, y] == [250, 170, 30]).all():
                    transformed_img[x, y] = 0
                elif (transformed_img[x, y] == [0, 0, 142]).all():
                    transformed_img[x, y] = 1
                elif (transformed_img[x, y] == [102, 102, 156]).all():
                    transformed_img[x, y] = 2
                elif (transformed_img[x, y] == [220, 20, 60]).all():
                    transformed_img[x, y] = 3
                elif (transformed_img[x, y] == [153, 153, 153]).all():
                    transformed_img[x, y] = 4
                elif (transformed_img[x, y] == [244, 35, 232]).all():
                    transformed_img[x, y] = 5
                elif (transformed_img[x, y] == [70, 70, 70]).all():
                    transformed_img[x, y] = 6
                elif (transformed_img[x, y] == [70, 130, 180]).all():
                    transformed_img[x, y] = 7
        transformed_img = transformed_img[:, :, 0:1]
        masks_tab.append(transformed_img)

    images_tab = np.array(images_tab)
    masks_tab = np.array(masks_tab)
    return images_tab, masks_tab

In [10]:
images, masks = load_data(df_train)
images_val, masks_val = load_data(df_val)
print("Nombre total d'images chargées:", len(images))
print("Nombre total de masks chargées:", len(masks))
print("Nombre total d'images de validation chargées:", len(images_val))
print("Nombre total de masks de validation chargées:", len(masks_val))
print("Shape images :", images.shape)
print("Shape masks :", masks.shape)
print("Shape images_val :", images_val.shape)
print("Shape masks_val :", masks_val.shape)

Nombre total d'images chargées: 5950
Nombre total de masks chargées: 5950
Nombre total d'images de validation chargées: 1000
Nombre total de masks de validation chargées: 1000
Shape images : (5950, 256, 256, 3)
Shape masks : (5950, 256, 256, 1)
Shape images_val : (1000, 256, 256, 3)
Shape masks_val : (1000, 256, 256, 1)


In [11]:
X_train = images.copy()
X_val = images_val.copy()
y_train = masks.copy()
y_val = masks_val.copy()

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
model = unet_model(input_shape, num_classes)
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model_X.keras', monitor='val_loss', save_best_only=True, verbose=1)

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    batch_size=32, 
                    epochs=100, 
                    callbacks=[early_stopping, model_checkpoint], 
                    verbose=1)

Epoch 1/100
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 67s/step - accuracy: 0.4931 - loss: 2.1748  
Epoch 1: val_loss improved from inf to 0.78218, saving model to model_X.keras
186/186 ━━━━━━━━━━━━━━━━━━━━ 12925s 69s/step - accuracy: 0.4938 - loss: 2.1702 - val_accuracy: 0.7580 - val_loss: 0.7822
Epoch 2/100
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 66s/step - accuracy: 0.8020 - loss: 0.6440  
Epoch 2: val_loss improved from 0.78218 to 0.62846, saving model to model_X.keras
186/186 ━━━━━━━━━━━━━━━━━━━━ 12859s 69s/step - accuracy: 0.8021 - loss: 0.6438 - val_accuracy: 0.8080 - val_loss: 0.6285
Epoch 3/100
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 67s/step - accuracy: 0.8410 - loss: 0.5216  
Epoch 3: val_loss improved from 0.62846 to 0.56648, saving model to model_X.keras
186/186 ━━━━━━━━━━━━━━━━━━━━ 12920s 69s/step - accuracy: 0.8410 - loss: 0.5216 - val_accuracy: 0.8315 - val_loss: 0.5665
Epoch 4/100
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 66s/step - accuracy: 0.8627 - loss: 0.4535  
Epoch 4: val_loss improved from 0.56648